In [ ]:
"""
Zachary Katz
zachary_katz@mines.edu
10 January 2026

Download seismic data hosted on IRIS using Obspy's fdsn client mass downloader.
This example downloads one station in one-day chunks for three days, and the 
stations's associated metadata, but other combinations/wildcards are possible.

Example usage from Obspy docs:
https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#module-obspy.clients.fdsn.mass_downloader 
"""

# Imports
import os
import obspy
from obspy.clients.fdsn.mass_downloader import Restrictions, MassDownloader, GlobalDomain

# User defined variables
# Network and station parameters are pulled from https://ds.iris.edu/mda/XS/?timewindow=2023-2023
data_center = "IRISPH5"
network = "XS"
station = "1001"
location = "*" # Wildcard, clicking on station shown '--' for location code
channel = "GPZ" # Can get all channels with, e.g., 'GP?'

start_time = obspy.UTCDateTime(2023, 1, 12)
days_to_download = 1
chunking = 0.5 # Chunk in half-day increments

mseed_storage = './_Data/mseed/'
stationxml_storage = './_Data/stationxml/'

In [ ]:
# Set up mass downloade
mdl = MassDownloader(providers=[data_center])
domain = GlobalDomain()

SEC_TO_HR = 3600
HR_TO_DAY = 24
end_time = start_time + days_to_download*SEC_TO_HR*HR_TO_DAY
restrictions = Restrictions(
    starttime=start_time,
    endtime=end_time,
    chunklength_in_sec=chunking*SEC_TO_HR*HR_TO_DAY,
    network=network, station=station, location=location, channel=channel,
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0
    )

mdl.download(domain, restrictions, mseed_storage=mseed_storage, stationxml_storage=stationxml_storage)


In [10]:
# Load and dayplot
st = obspy.read(mseed_storage + f'{network}.{station}..{channel}*.mseed')
outdir = './_output/'
os.makedirs(outdir, exist_ok=True)
st.plot(type="dayplot", interval=60, color='black', outfile=f"{outdir}/Dayplot.png" ) # Interal in mins
